In [1]:
!pip --version

# To install pytorch -------------------------------------------------------------------------------------
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# To install Tensorflow ----------------------------------------------------------------------------------
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

# To install Flax ----------------------------------------------------------------------------------------
!pip install flax

# or to install the latest version of Flax:
!pip install --upgrade git+https://github.com/google/flax.git

# Set up transformers
!pip install git+https://github.com/huggingface/transformers

# Check if installed
!python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"

!pip install transformers[torch]

!pip install accelerate -U

!pip install -q datasets peft evaluate

!pip install opacus

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Cloning https://github.com/google/flax.git to /tmp/pip-req-build-lhtzwx_f
  Running command git clone --filter=blob:none --quiet https://github.com/google/flax.git /tmp/pip-req-build-lhtzwx_f
  Resolved https://github.com/google/flax.git to commit 85dfad242e56098849dbf05e7e4657b3a40820f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for flax: filename=flax-0.8.0-py3-none-any.whl size=635291 sha256=fd33d50b980f85452627e84bc308778f7bd269c3704bc1ba4022e653bb54723f
  Stored in directory: /tmp/pi

In [11]:
from transformers import (AutoModelForSequenceClassification,AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer )
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    PromptTuningConfig,
)
from peft import ( get_peft_config, get_peft_model, get_peft_model_state_dict, set_peft_model_state_dict, PeftModelForSeq2SeqLM, PeftType,PromptEncoderConfig,PeftModelForSequenceClassification,PrefixTuningConfig, TaskType)
from peft import PromptEmbedding, PromptTuningConfig
from datasets import load_dataset
import evaluate
from transformers import AdamW
import torch
import numpy as np
import pandas as pd
from peft import LoraModel, LoraConfig
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from datasets import Dataset
import opacus
from opacus import PrivacyEngine
from opacus.grad_sample import GradSampleModule
from opacus.validators import ModuleValidator


dataset = load_dataset("glue", "mnli")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=3) # as output 0 or 1
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", padding_side = "right")
model = model.to('cuda')
# optimizer = AdamW(model.parameters(), lr= 0.005, eps = 1e-8)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Assuming the dataset has a 'train' split, modify this according to your dataset's splits
data_split = dataset['train']

# Convert the dataset split to a pandas DataFrame for easier splitting
df = data_split.to_pandas()
df = df[0:150000]

# Split the dataset into train and test sets using train_test_split from sklearn
# df1, df2 = train_test_split(df, test_size = 0.5, random_state = 42)
train_df, test_df = train_test_split(df , test_size=0.2, random_state=42)

# Convert the splits back to datasets
train_dataset = train_df.reset_index(drop=True)
test_dataset = test_df.reset_index(drop=True)

train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

x_train1 = list(train_dataset["premise"])
x_train2 = list(train_dataset["hypothesis"])
y_train = list(train_dataset["label"])

x_test1 = list(test_dataset["premise"])
x_test2 = list(test_dataset["hypothesis"])
y_test = list(test_dataset["label"])

X_train_tokenized = tokenizer(x_train1, x_train2, padding=True, truncation=True, max_length = 128)
# X_val_tokenized = tokenizer(x_validation, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(x_test1, x_test2, padding=True, truncation = True, max_length = 128)


class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to('cuda') for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx]).to('cuda')
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
# val_dataset = Dataset(X_val_tokenized, y_validation)
test_dataset = Dataset(X_test_tokenized, y_test)

train_dataloader = DataLoader(train_dataset, batch_size = 1024, drop_last = True) # WIth priv batch size to 1024
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size = 1024, drop_last = True)
# val_dataloader = DataLoader(val_dataset, batch_size = 500)

In [13]:
config = PrefixTuningConfig(
    peft_type="PREFIX_TUNING",
    task_type="TEXT_PAIR",
    num_virtual_tokens= 50, # 50 for privacy 20 non priv
)

# model = LoraModel(model, config, "default")

model = PeftModelForSequenceClassification(model, config)

# model = get_peft_model(model, config)
model.print_trainable_parameters()

model = ModuleValidator.fix(model)
model = GradSampleModule(model)

# l = []

optimizer = AdamW(model.parameters(), lr= 0.005, eps = 1e-8) # 0.005 in case of privacy 0.001 in non

trainable params: 25,987 || all params: 4,412,294 || trainable%: 0.5889680062117347


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
!pip install --upgrade peft

In [5]:
model.train()
for batch in train_dataloader:
  input_ids = batch['input_ids']
  attention_mask = batch['attention_mask']
  labels = batch['labels']

  outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
  break
outputs


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


SequenceClassifierOutput(loss=tensor(1.1439, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[-0.1948,  0.0606, -0.2995],
        [-0.1546,  0.1238, -0.5139],
        [-0.0570,  0.0277, -0.3990],
        ...,
        [-0.2246,  0.2599, -0.5042],
        [-0.0867,  0.2059, -0.3934],
        [ 0.0803,  0.1189, -0.3294]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [6]:
CUDA_LAUNCH_BLOCKING=1

In [14]:

model.train()
privacy_engine = PrivacyEngine()
model, optimizer, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_dataloader,
    target_delta= 1 / len(train_dataloader),
    target_epsilon=  8,
    epochs= 3,
    max_grad_norm = 0.05,
)



/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)


In [ ]:
def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()
    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:

        with torch.no_grad():
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss, logits = outputs[:2]

            log = logits.cpu()
            log = log.numpy()

            preds = np.argmax(log, axis=1)
            labels = batch['labels'].detach().cpu().numpy()

            loss_arr.append(loss.item())
            accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

model.train()

# Training loop
optimizer.zero_grad()  # Explicitly zero the gradient buffers

for epoch in range(60):  # Number of epochs
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        logits = outputs[1]
        predictions = torch.argmax(logits, dim = -1)

    eval_loss, eval_accuracy = evaluate(model)
    print("eval loss",eval_loss)
    print("accuracy: ",eval_accuracy)
    print("ends")

    model.train()  # Set the model back to training mode


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1324: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "


eval loss 1.0984437712307633
accuracy:  0.3480940193965517
ends
eval loss 1.1094341278076172
accuracy:  0.33741918103448276
ends
eval loss 1.1030463884616721
accuracy:  0.35051858836206895
ends
eval loss 1.1245030329145234
accuracy:  0.3483970905172414
ends
eval loss 1.101011555770348
accuracy:  0.353515625
ends
eval loss 1.0988241762950504
accuracy:  0.3541554418103448
ends
eval loss 1.1002294852815826
accuracy:  0.34880118534482757
ends
eval loss 1.1060315370559692
accuracy:  0.34967672413793105
ends
eval loss 1.0999207250003158
accuracy:  0.3578596443965517
ends
eval loss 1.096732731523185
accuracy:  0.3569841056034483
ends
eval loss 1.10957250923946
accuracy:  0.35129310344827586
ends
eval loss 1.099708236497024
accuracy:  0.35651266163793105
ends
eval loss 1.1099530170703757
accuracy:  0.35651266163793105
ends
eval loss 1.098783801341879
accuracy:  0.35509832974137934
ends
eval loss 1.1162848719235123
accuracy:  0.35075431034482757
ends
eval loss 1.1000412949200333
accuracy:  0.35